In [21]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tifffile as tf
from scipy.ndimage.filters import gaussian_filter1d as gaussian
from glob import glob
from scipy.signal import savgol_filter, medfilt
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit
import os

In [4]:
# 1. load data into pandas data frame

# 2. convert data from I(t) to c_i(t) via equation for G quadruplex binding
    # - find t0
    # - 
# 3. calculate dc_i/dt for each GUV

# 4. calculate background intensity and convert this to c_0(t) via g quadruplex binding model

# 5. plot dc_/dt/A_GUV vs c_0-c_i (expect to see a positive linear relationship)


# as data includes actual G4 background for GUV permeability and later Gramicidin A experiments, start with these experiments


In [2]:
App = 3.16 #um^2
Amin= 75*App*1e-12 #m^2


# G quadruplex model binding



def exp_I(t,k,c):
    
    k_b = 8.8
    A = 1
#     c = 0.98
#     c = 0.46
    return c+(1-c)/(A*k_b*(1-np.exp(-k*t))**2+1)


def inv(I, k = 8.8,A = 0.5):
    
    if I > 1:
        return 0
    
    elif I < A:
        return 1
        
    else:
        return np.sqrt((1/k)*((1-A)/(I-A)-1))

#smoothing configs

sigma = 30




# Calibration model fit


## **If you have already fitted a calibration function and saved the parameters skip this section**

## Discussion of binding model deficiencies

1. **Binding model**

By considering the binding reaction G4 + nK <=> complex we can develop a model to fit the calibration experiment for the g quadruplex sensor.

It was found that in an excess of potassium the 'completely folded' g quadruplex goes to 0.5 of its free intensity. I assume then that in the intermediate concentrations of potassium, there is a mixture of unfolded and folded g quadruplex. 

Therefore $I = I_0[G4]_{free} + 0.5I_0[G4]_{folded}$,

by assuming on the addition of potassium the system reaches equilibrium much faster than the timescales of the transport, then we gain  $\frac{I}{I_0} = 0.5(\frac{1}{1+K_b[K]^2} +1)$

$K_b$ is found by fitting this function to the calibration.

2. **Problems**

As this function plateaus at 0.5, defining a sensible inverse that can practicably applied to a noisy intensity measurement (to measure [K]  is difficult. As noisy variations near $\frac{I}{I_0}$ cause wild swings in the concentration.

3. **Solution**

Therefore we surpress these swings by defining an approximate inverse that does not diverge at the edges.

We do this by creating an effective intensity of the folded g quadruplex that is artificially lower than the believe value. Therefore our calibration model becomes:

$I = I_0[G4]_{free} + c_{eff}I_0[G4]_{folded}$, where $c_{eff} < 0.5$.
    
$\frac{I}{I_0} = c_{eff} + \frac{1-c_{eff}}{K_b[K]^2 +1}$

**This doesn't really work as we still have a blowing up value, need to define a nicer function**

Define a polynomial approximation that is similar

In [4]:
# finding a suitable calibration

calib_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/RelIntensG41Concentration.csv')

In [9]:
%matplotlib qt
I= np.linspace(1,0.2,100)

fig, ax = plt.subplots(1,1)

ax.scatter(calib_data['Rel. Intes'],calib_data['K conc [mM]'],marker = '+',s = 5)

legend = []

# fit a polynomial


fit = np.polyfit(-np.log(calib_data['Rel. Intes'].iloc[:-1]),calib_data['K conc [mM]'].iloc[:-1],3)

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
#         print(len(fit)-i)
    return c



ax.plot(I,fit_func(I,fit))

legend.append('Calibration')

def model_fit(cK,kb,c):
    return c + (1-c)/(kb*cK**2+1)

params = curve_fit(model_fit,calib_data['K conc [mM]'],calib_data['Rel. Intes'], p0 = [8.8,0.45])

cK_bar = np.linspace(0,2)
I_bar = model_fit(cK_bar,params[0][0],params[0][1])

ax.set_xlim([0.35,1.05])
ax.plot(I_bar,cK_bar,label = 'Model')
legend.append('Model')

l  = plt.errorbar(calib_data['Rel. Intes'],calib_data['K conc [mM]'],rel_errorK*calib_data['K conc [mM]'],drelI,c = 'k')

ax.set_xticks([0.4,0.6,0.8,1])
ax.set_xticklabels([0.4,0.6,0.8,1],fontsize  = 15)

ax.set_yticks([0,1,2])
ax.set_yticklabels([0,1,2],fontsize = 15)
ax.set_ylim([-0.1,2.1])
plt.tick_params(direction = 'in',top = True,bottom = True,right = True, left = True,length = 5)
ax.set_xlabel('Rel Intensity [a.u]',fontsize = 18)
ax.set_ylabel('[K]/[mM]',fontsize = 18)
l[0].remove()
plt.legend(legend)



3
2
1
0


In [30]:
help(l.remove)

Help on method remove in module matplotlib.container:

remove() method of matplotlib.container.ErrorbarContainer instance



# Errors in calibration

**Errors are mainly due to the focal plane measured**


In [6]:
raw_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/1uM/Intensities0-2mM.csv')

In [7]:
n = 512**2

print(raw_data['StdDev']/np.sqrt(n))

dI = raw_data['StdDev']/np.sqrt(n)
drelI = dI/raw_data['Mean']

drelI = drelI*10

0    2.244234
1    2.140793
2    1.921438
3    1.709535
4    1.638162
5    1.617246
Name: StdDev, dtype: float64


# Potassium concentration errors

In [8]:
rel_errorK = 0.02



# Load calibration function parameters and define function



In [3]:
fitsdf = pd.read_csv('../Calibrations/Calibration_approx_fit_params.csv')

fit = fitsdf['Fitting Params']

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
#         print(len(fit)-i)
    return c


# Single GUV modelling

In [4]:
t0 = 50
t1 = 300
dt = 15/60

In [23]:
# load 260pM Gramicidin A experiment
DATA_DIR = '/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/3nM/'
dfI = pd.read_csv(os.path.join(DATA_DIR,'collatedIabs.csv'))
dfA = pd.read_csv(os.path.join(DATA_DIR,'collatedAabs.csv'))

In [24]:

dfI = dfI.iloc[:,1:]
dfA = dfA.iloc[:,1:]

In [25]:
dfA = dfA.fillna(0)
dfI = dfI.fillna(0)

# Remove small detected objects, most likely not GUVs

**check that the size is appropriate for the experiment**

In [26]:
# remove traces for GUVs below a certain area

for col in dfI.columns:
    
    if col in dfA.columns:
        area = 1e-12*App*np.mean(dfA[col].iloc[:30])
        
        if area < Amin:
            dfI.pop(col)
            dfA.pop(col)

# Remove intensity curves with sudden drops
## These are likely from GUVs suddenly escaping the trap

In [27]:

def remove_burst_events(data,threshold = 0.45):
    
    for column in data.columns:
        
        if len(data[column][data[column] < threshold]) >=1:
            data.pop(column)
            

In [29]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(dfI[col])

In [28]:
# normalise intensities

for col in dfI.columns:
    dfI[col] = dfI[col]/np.mean(dfI[col][:30])
remove_burst_events(dfI)

# Smooth data

## Smoothing strategy

1. In experiments where there was relatively fast decay in intensity initially, but slows down (as exponential decays do..), yet there is still a lot of noise, it may be necessary to smooth different regions with different 'cutoff's.
When the intensity has dropped, the signal to noise drops. What's more during the slower period of intensity decay, there is increasing likelihood that some long timescale event will have happened to distort the data. This long timescale noise may be of a similar timescale to the 'signal' in the early period of rapid decay. 

2. The **solution** chosen here, is to have the user select a timepoint to bisect the time traces. The early period up until that timepoint is smoothed with the 1st user input smoothing parameter (usually between 11-21). Then the later period of slow decay is smoother with much larger smoothing settings (50-80). In order to ensure a smooth 'stitching' of these differently smoothed traces, we concatenate and repeat the smoothing with the smallest of the two specified smoothing parameters.

In [30]:
def smooth_decay(I,t0,t1=-1,smoothing_0 = 51,smoothing_1 = 81):
    
    
    if smoothing_0 > t1-t0 and t1 != -1:
        raise ValueError('Smoothing parameter cannot be greater than input data')
    
    smoothed_decay_0 = I.iloc[t0:t1]
    
    smoothed_decay_0 = savgol_filter(smoothed_decay_0,smoothing_0,1)
    
    if t1 != -1:
        
        smoothed_decay_1 = I.iloc[t1:]
        smoothed_decay_1  = savgol_filter(smoothed_decay_1,smoothing_1,1)
        
        smoothed_decay = np.concatenate((smoothed_decay_0,smoothed_decay_1))
    else:
        smoothed_decay = smoothed_decay_0
                               
    final_smoothed_decay = savgol_filter(smoothed_decay,smoothing_0,1)
    return final_smoothed_decay
                                   

In [33]:
#Check smoothing still fits data
#
num = 8
fig,axs = plt.subplots(1,num)

for i,col in enumerate(dfI.columns[np.random.randint(0,len(dfI.columns),num)]):
    #
    axs[i].plot(dfI[col].iloc[t0:].to_numpy())
    
    axs[i].plot(smooth_decay(dfI[col],t0,t1,smoothing_0=31,smoothing_1=101))
    

# Use calibration inverse to convert I to [K]

In [34]:
dfC_dict = {}

# can't exploit numpy broadcasting

for col in dfI.columns:
    
    smoothed_decay = smooth_decay(dfI[col],t0,t1,smoothing_0 = 31,smoothing_1=101)
#     print(smoothed_decay)
#     normalised_sd = smoothed_decay/np.mean(smoothed_decay[:30])
    
    cs = []
    for I in smoothed_decay:
#         print(I)
        cs.append(fit_func(I,fit))
    
    
    dfC_dict[col] = cs
    
dfC = pd.DataFrame(dfC_dict)

In [14]:
%matplotlib qt

fig, axs = plt.subplots(1,3)

t = dt*np.arange(0,len(dfC))
for col in dfC.columns:
    
    
    axs[0].plot(t,dfC[col])
#     axs[1].plot(t,medfilt(dfI[col].iloc[t0:t1],31))
    axs[2].plot(t,dfI[col].iloc[t0:])

    for ax in axs:
        ax.set_ylim([0,1])



In [13]:
# find representative background concentration variation

# tifs = glob('/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/260pM/*.tif')

# bgs = []

# fig, ax = plt.subplots(1,1)

# for tif in tifs:
    
#     vid = tf.TiffFile(tif)
    
#     frames = vid.asarray()
    
#     bg = np.median(frames,axis  =(1,2))
#     bg = np.array(bg)/np.max(bg)
#     bgs.append(bg)
    
    
#     ax.plot(bg)

# This confirms that the normalised background is the same for each chamber. Suggesting potassium concs are identical between chambers



In [ ]:
# # filter concentration data by gradient
# # if 

# fig, axs = plt.subplots(1,2)

# for col in dfC.columns:
    
#     grad = np.gradient(dfC[col])
    
#     if len(grad[grad <= -0.0009])/len(grad) > 0.05:
#         dfC.pop(col)
#         continue
#     axs[0].hist(grad,15)
#     axs[1].plot(dfC[col])

In [35]:
tif = '/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/3nM/20210920_GramA_260pM07_3nM815_1mMKCl_1_MMStack_Pos10.ome.tif'


vid = tf.TiffFile(tif)

frames = vid.asarray()
bg = np.median(frames[:,455:,435:],axis = (1,2))
final_g4 = np.mean(bg[750:])
initialbg = np.mean(bg[241:260])

bg = bg[210:810]
bg = bg-final_g4
bg = bg/np.max(bg)
bg_k = 1- bg
bg= pd.DataFrame({'Bg': bg})
smoothed_bg = smooth_decay(bg['Bg'],0,200,smoothing_0=19,smoothing_1=91)  

OME series: not an ome-tiff master file


In [37]:
len(dfC),

(550,)

In [38]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(np.arange(len(dfC[col])),dfC[col])
    
t_0_record = 50
ax.plot(np.arange(len(bg_k[t_0_record:])),bg_k[t_0_record:])


In [63]:
# fit for smoother bgK

t = np.arange(-100*60,100*60,1)
from scipy.special import erf
# parameters

D1 = 4e-10 #m^2s^-1
D2 = 1e-9 #m^2s^-1

Q = 20e-9/3600 #m^3s-1
A = (30e-6)*(800e-6)
R_tube = 1.5e-3 #m
L_tube = 0.15 #m
V = np.pi*R_tube**2*L_tube

def erf_input(x,D,k,t_c):
    return (Q*(x-t_c)/A)/(np.sqrt(4*np.pi*k*D*V/Q))


z1 = erf_input(t,D1,0,1)
z2 = erf_input(t,D2,0,1)
y1 = 0.5*(1+erf(z1))
y2 = 0.5*(1+erf(z2))

def background(x,k,t_c):
    D = 4e-10 #m^2s^-1
    z = erf_input(x,D,k,t_c)
    
    return_val = 0.5*(1+erf(z))
    
    return return_val

/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [56]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(np.arange(len(dfC[col])),dfC[col])
    
    
t_0_record = 40
ax.plot(np.arange(len(approx_k_conc[t_0_record:])),approx_k_conc[t_0_record:])

In [87]:

# estimate the offset time by the 0.5 point time
# the relative diffusion to ideal diffusion constant needs to be estimated with experience

p0 = [6e-3,18]

t = dt*np.arange(bg_k.shape[0])
t_fit = list(t[50:])
bg_k_fit = bg_k[50:]
params = curve_fit(background,t_fit,bg_k_fit,p0)


/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in sqrt


In [88]:
params

(array([8.53472067e-04, 1.50730910e+01]),
 array([[ 7.75015514e-09, -1.48876744e-06],
        [-1.48876744e-06,  7.80555376e-03]]))

In [94]:
# t_bar = np.arange(len(bg_k))
# approx_k_conc = background(t_bar,params[0][0]*8.5,params[0][1]+3)
# fig,ax = plt.subplots(1,1)

ax.plot(bg_k[t_0_record:])
ax.plot(approx_k_conc[:-t_0_record])

In [92]:
params[0][0] = 8.5*params[0][0]
params[0][1] = params[0][1]+3

In [93]:
t_bar = np.arange(len(bg_k))
approx_k_conc = background(t_bar,params[0][0],params[0][1])

In [23]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(np.arange(len(dfC[col])),dfC[col])
    
t_0_record = 50
ax.plot(np.arange(len(approx_k_conc[t_0_record:])),approx_k_conc[t_0_record:])


In [131]:
# c_bg = []
# for x in bg[360+t0:]:
#     c_bg.append(inv(x))

In [132]:
# fig, axs = plt.subplots(1,2)


# smoothed_background = gaussian(c_bg,sigma)
# axs[0].plot(smoothed_background)
# axs[1].plot(bg[360+t0:])

In [80]:
# drift in the background intensity over time, once the potassium has arrived, is causing a drifting predicted potassium concentration
# this is unlikely to be the case in reality.
# Therefore the best thing may be to just assume that after an initial 80 frame boundary region (20 mins) for this exp
# the potassium concentration externally reaches 1mM.


In [102]:
# calculating c_ext - c_i
t_0_record = 50

dfdeltaC_dict = {}

for col in dfC:
    dfdeltaC = []
    for i in range(len(dfC[col])):
        
        dfdeltaC.append(approx_k_conc[t_0_record:][i] - dfC[col].iloc[i])
    
    dfdeltaC_dict[col] = dfdeltaC
dfdeltaC = pd.DataFrame(dfdeltaC_dict)

In [96]:
dfdeltaC

,11003,11004,11006,11009,11010,11013,11017,11018,11021,11022,...,15004,15006,15009,15011,15013,15014,15016,15017,15018,15023
0,1.020981,0.987056,1.003956,1.004282,1.009440,1.031160,1.032054,1.016630,1.009984,1.024309,...,0.989770,1.008047,0.990025,1.005871,1.000669,1.008185,1.023798,1.006246,1.011421,1.038546
1,1.019838,0.987459,1.004246,1.004465,1.008945,1.031088,1.031795,1.016224,1.009742,1.024211,...,0.989989,1.007634,0.990556,1.005720,0.999832,1.008181,1.023021,1.006155,1.011616,1.038240
2,1.018691,0.987855,1.004530,1.004643,1.008443,1.031010,1.031530,1.015812,1.009495,1.024106,...,0.990203,1.007215,0.991081,1.005564,0.998990,1.008172,1.022240,1.006058,1.011805,1.037929
3,1.017542,0.988249,1.004810,1.004816,1.007938,1.030928,1.031262,1.015397,1.009244,1.023998,...,0.990413,1.006793,0.991602,1.005404,0.998145,1.008158,1.021455,1.005957,1.011990,1.037614
4,1.016392,0.988640,1.005088,1.004988,1.007431,1.030844,1.030991,1.014981,1.008991,1.023887,...,0.990620,1.006369,0.992122,1.005242,0.997299,1.008142,1.020669,1.005853,1.012173,1.037296
5,1.015243,0.989030,1.005365,1.005158,1.006923,1.030758,1.030719,1.014562,1.008736,1.023775,...,0.990826,1.005944,0.992641,1.005078,0.996452,1.008125,1.019883,1.005749,1.012355,1.036978
6,1.014095,0.989419,1.005641,1.005328,1.006415,1.030672,1.030446,1.014144,1.008481,1.023662,...,0.991032,1.005518,0.993159,1.004914,0.995606,1.008107,1.019097,1.005643,1.012536,1.036659
7,1.012949,0.989808,1.005917,1.005497,1.005907,1.030585,1.030173,1.013725,1.008225,1.023549,...,0.991237,1.005092,0.993678,1.004749,0.994761,1.008089,1.018311,1.005538,1.012716,1.036339
8,1.011804,0.990197,1.006193,1.005665,1.005398,1.030498,1.029899,1.013306,1.007970,1.023436,...,0.991442,1.004666,0.994196,1.004585,0.993916,1.008071,1.017525,1.005432,1.012897,1.036020
9,1.010661,0.990587,1.006469,1.005834,1.004890,1.030411,1.029626,1.012887,1.007714,1.023322,...,0.991647,1.004240,0.994715,1.004420,0.993073,1.008052,1.016741,1.005326,1.013077,1.035700


In [97]:
fig, axC = plt.subplots(1,1)

delete_cols = []

for col in dfdeltaC.columns:
    graddeltaC = np.gradient(dfdeltaC[col])
    
    
#     if len(graddeltaC[graddeltaC > 0]) > 100:
#         delete_cols.append(col)
#         continue
        
    axC.plot(dfdeltaC[col])

/home/marcus/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [24]:
for col in delete_cols:
    dfdeltaC.pop(col)

In [103]:
# now calculate J

# J = (1/A)*dN/dt = R/3* dc_i/dt

dfJ_dict = {}


for col in dfdeltaC:
    
    if col in dfA:
        print(col)
        delta_c = np.gradient(dfC[col],dt)/60 #molm^-3s^-1
        print(dfC[col])
        A = np.mean(dfA[col].iloc[:30])*App*1e-12 #m^2
#         if A < Amin:
#             continue
        R = np.sqrt(A/np.pi) #m
        
        dfJ_dict[col] = R/3*delta_c #molm^-2s^-1
        

dfJ = pd.DataFrame(dfJ_dict)




11003
0        -0.02101867876465714
1        -0.01986035717191154
2       -0.018703903246096065
3       -0.017549316162802597
4         -0.0163965950874957
5        -0.01524573917546071
6         -0.0140967475717516
7        -0.01294961941114189
8       -0.011804353818071985
9       -0.010660949906596296
10      -0.009519406780333351
11      -0.008379723532412037
12      -0.007241899245419745
13      -0.006105932991348095
14      -0.004971823831542008
15      -0.003839570816645442
16     -0.0028003214152358554
17     -0.0017872847692542273
18      -0.000839342533492821
19      6.221289538381816e-05
20      0.0009439893504926125
21      0.0017696425155965673
22      0.0026747693944724187
23      0.0034311169890142053
24       0.004187600974061981
25       0.004847196312860067
26       0.005428913230470852
27       0.006035327880164775
28       0.006619492284372196
29        0.00711980579000356
                ...          
520       0.06316052584417113
521         0.063069513131208
522 

In [22]:
fig, axs = plt.subplots(1,3)
legend = []

for col in dfJ.columns:

    legend.append(col)
#     print(dfdeltaC)

    axs[0].scatter(dfdeltaC[col],dfJ[col]*1e10, marker = '+', s = 10)
    print(dfJ[col])

    axs[1].plot(dt*np.arange(0,len(dfC[col])),dfC[col])

    
#     gradients = []
#     win = 10
#     for i in range(dfJ[col].shape[0]-(win-1)):
        
#         gradient = np.mean(np.gradient(dfJ[col].iloc[i:i+win]))
#         gradients.append(gradient)
        
    
    axs[2].plot(np.gradient(dfJ[col],dfdeltaC[col]))
axs[0].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[1].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[0].set_xlabel('[K_ext]-[K_int] [mM]',fontsize = 18)

axs[0].set_ylabel('J [molm^-2s^-1]',fontsize = 18)

axs[1].set_ylabel('[K_int] [mM]',fontsize = 18)


axs[1].set_xlabel(' T [mins]',fontsize = 18)


#xticks

axs[0].set_xticks([0,0.5,1])
axs[0].set_xticklabels([0,0.5,1],fontsize = 15)
axs[0].set_xlim([0,1.05])

axs[1].set_xticks([0,60,120])
axs[1].set_xticklabels([0,60,120],fontsize = 15)

#yticks

axs[0].set_yticks([0,0.01,0.02])
axs[0].set_yticklabels([0,0.01,0.02],fontsize = 15)

axs[0].set_ylim([-0.0005,0.0105])
axs[1].set_yticks([0, 0.5,1])
axs[1].set_yticklabels([0,0.5,1],fontsize = 15)


axs[0].set_ylim([-0.0005,0.0205])
axs[1].set_ylim([-0.05,1.05])


plt.legend(legend)
# plt.rcParams['text.usetex'] = True

# def myticks(x,pos):

#     if x == 0: return "$0$"

#     exponent = int(np.log1

#     coeff = x/10**exponent

#     return r"${:2.0f} \times 10^{{ {:2d} }}$".format(coeff,exponent)

# axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(myticks))

0     -6.194001e-10
1     -6.196731e-10
2     -6.202192e-10
3     -6.207656e-10
4     -6.213121e-10
5     -6.218588e-10
6     -6.224057e-10
7     -6.229527e-10
8     -6.234999e-10
9     -6.671621e-10
10    -7.266336e-10
11    -6.684567e-10
12    -5.974885e-10
13    -5.445802e-10
14    -4.571183e-10
15    -3.361335e-10
16    -2.130722e-10
17    -1.371179e-10
18    -9.652303e-12
19     7.735025e-11
20    -3.529759e-11
21    -4.534398e-11
22     1.459296e-10
23     1.243547e-10
24    -6.772734e-12
25    -5.606538e-12
26     1.702743e-11
27     4.556855e-11
28     1.727364e-10
29     3.463584e-10
           ...     
454   -4.296798e-10
455   -4.357307e-10
456   -4.615231e-10
457   -4.928011e-10
458   -4.517343e-10
459   -4.297786e-10
460   -5.439470e-10
461   -6.753304e-10
462   -7.292492e-10
463   -7.744054e-10
464   -7.691267e-10
465   -7.456559e-10
466   -6.900822e-10
467   -5.941854e-10
468   -5.119193e-10
469   -3.916256e-10
470   -2.953466e-10
471   -2.843838e-10
472   -3.554253e-10


0      2.135854e-09
1      2.132620e-09
2      2.126156e-09
3      2.119699e-09
4      2.113249e-09
5      2.106806e-09
6      2.100371e-09
7      2.093944e-09
8      2.087525e-09
9      2.080050e-09
10     1.972977e-09
11     1.727344e-09
12     1.433686e-09
13     1.229859e-09
14     1.094768e-09
15     1.001594e-09
16     1.054070e-09
17     9.440535e-10
18     7.350522e-10
19     6.913714e-10
20     9.722272e-10
21     1.384044e-09
22     1.446013e-09
23     1.268136e-09
24     1.067645e-09
25     8.347759e-10
26     7.778883e-10
27     9.715160e-10
28     8.317685e-10
29     6.470244e-10
           ...     
454    6.439563e-10
455    1.109224e-09
456    5.300466e-10
457    6.562781e-11
458    3.145492e-10
459    1.474810e-10
460   -1.784352e-10
461   -7.616298e-12
462   -1.513713e-10
463   -3.753445e-10
464   -5.865982e-10
465   -1.120090e-09
466   -1.392736e-09
467   -1.623077e-09
468   -1.940012e-09
469   -2.686812e-09
470   -3.634556e-09
471   -4.256831e-09
472   -4.701671e-09


0     -5.621560e-10
1     -5.625981e-10
2     -5.634845e-10
3     -5.643749e-10
4     -5.652693e-10
5     -5.661677e-10
6     -5.670700e-10
7     -5.679763e-10
8     -5.688864e-10
9     -5.922982e-10
10    -5.153358e-10
11    -6.054889e-10
12    -1.047647e-09
13    -1.171009e-09
14    -9.417359e-10
15    -8.089172e-10
16    -3.715809e-10
17     1.170049e-10
18     2.591566e-10
19     6.238980e-10
20     9.957864e-10
21     4.568681e-10
22    -7.172773e-10
23    -9.998508e-10
24    -1.015442e-09
25    -7.139145e-10
26    -4.173279e-10
27    -7.083322e-10
28    -7.703191e-10
29    -5.909974e-10
           ...     
454   -2.095296e-09
455   -2.116750e-09
456   -1.853461e-09
457   -1.159250e-09
458   -4.298391e-10
459   -2.426728e-11
460    6.724039e-11
461   -2.537041e-11
462   -8.557626e-11
463    2.550303e-10
464    5.957349e-10
465    6.266387e-10
466    8.080905e-10
467    9.728137e-10
468    1.024719e-09
469    1.135803e-09
470    1.279594e-09
471    1.345264e-09
472    1.416933e-09


0     -7.165409e-10
1     -7.169916e-10
2     -7.178932e-10
3     -7.187953e-10
4     -7.196978e-10
5     -7.206007e-10
6     -7.215040e-10
7     -7.224077e-10
8     -7.233118e-10
9     -6.539599e-10
10    -5.465022e-10
11    -5.125868e-10
12    -4.320801e-10
13    -3.128127e-10
14    -2.304033e-10
15    -1.605721e-10
16    -8.641943e-11
17     3.034677e-11
18     1.111445e-10
19     3.341720e-11
20    -2.005568e-11
21    -1.168376e-11
22     4.728379e-12
23     7.305032e-11
24     9.817535e-11
25     1.699859e-10
26     1.152705e-10
27     4.596503e-11
28     3.683206e-11
29    -4.112212e-11
           ...     
454    3.715370e-10
455    3.393207e-10
456    3.865978e-10
457    4.677523e-10
458    5.054976e-10
459    5.348866e-10
460    5.726001e-10
461    5.377719e-10
462    4.217207e-10
463    2.621948e-10
464    2.077222e-10
465    1.627571e-10
466    6.304373e-11
467   -4.443458e-12
468   -9.987245e-11
469   -1.246284e-10
470   -1.190746e-10
471   -2.819819e-10
472   -5.014361e-10


/home/marcus/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1069: RuntimeWarning: divide by zero encountered in double_scalars
  a = -(dx2)/(dx1 * (dx1 + dx2))
/home/marcus/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1070: RuntimeWarning: divide by zero encountered in double_scalars
  b = (dx2 - dx1) / (dx1 * dx2)
/home/marcus/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1071: RuntimeWarning: divide by zero encountered in double_scalars
  c = dx1 / (dx2 * (dx1 + dx2))
/home/marcus/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1077: RuntimeWarning: invalid value encountered in double_scalars
  out[tuple(slice1)] = a * f[tuple(slice2)] + b * f[tuple(slice3)] + c * f[tuple(slice4)]


...

0      1.157890e-09
1      1.156748e-09
2      1.154463e-09
3      1.152180e-09
4      1.149898e-09
5      1.147618e-09
6      1.145339e-09
7      1.143062e-09
8      1.140787e-09
9      1.021672e-09
10     9.030405e-10
11     7.937238e-10
12     6.488525e-10
13     4.913499e-10
14     3.367992e-10
15     3.336186e-10
16     2.900799e-10
17     1.451186e-10
18     5.498803e-11
19     1.421681e-10
20     2.139522e-10
21     2.962353e-10
22     3.897957e-10
23     5.323089e-10
24     6.620700e-10
25     5.083835e-10
26     4.410342e-10
27     5.180540e-10
28     6.156703e-10
29     5.852482e-10
           ...     
454   -5.006452e-10
455   -4.479946e-10
456   -2.816444e-10
457   -2.198542e-10
458   -1.086478e-10
459    7.641321e-11
460    2.265499e-11
461   -5.403438e-11
462    9.336078e-12
463    1.503374e-10
464    3.506457e-10
465    4.645323e-10
466    4.687554e-10
467    5.543166e-10
468    6.667268e-10
469    7.776013e-10
470    9.592203e-10
471    1.036513e-09
472    1.053789e-09


0      7.151836e-10
1      7.149068e-10
2      7.143532e-10
3      7.137997e-10
4      7.132463e-10
5      7.126931e-10
6      7.121400e-10
7      7.115871e-10
8      7.110343e-10
9      6.004363e-10
10     5.484698e-10
11     5.651110e-10
12     5.474216e-10
13     5.057560e-10
14     4.172186e-10
15     4.186064e-10
16     3.941763e-10
17     2.956958e-10
18     2.432819e-10
19     2.065129e-10
20    -4.795612e-12
21    -2.045249e-10
22    -1.940341e-10
23    -8.281076e-11
24    -1.336880e-10
25    -1.058315e-10
26    -3.689302e-11
27    -6.173467e-11
28     1.533503e-11
29    -6.456669e-11
           ...     
454    3.631323e-10
455    1.475660e-10
456    2.042612e-11
457    2.186166e-11
458   -1.296481e-10
459   -9.004566e-11
460    1.129454e-10
461    1.302525e-10
462    7.274646e-11
463    1.170024e-10
464    2.737547e-10
465    2.707602e-10
466    1.875746e-10
467    1.167117e-10
468    1.268681e-10
469    9.311123e-11
470   -1.062928e-10
471   -2.273490e-10
472   -2.595777e-10


0      7.303627e-10
1      7.299898e-10
2      7.292439e-10
3      7.284983e-10
4      7.277529e-10
5      7.270077e-10
6      7.262628e-10
7      7.255181e-10
8      7.247736e-10
9      7.484732e-10
10     8.118433e-10
11     8.041423e-10
12     5.991244e-10
13     5.529451e-10
14     1.017193e-09
15     1.385211e-09
16     1.460734e-09
17     1.532963e-09
18     1.621262e-09
19     1.637177e-09
20     1.774479e-09
21     2.072266e-09
22     2.211145e-09
23     2.237491e-09
24     2.339620e-09
25     2.278655e-09
26     2.129785e-09
27     2.147434e-09
28     2.191951e-09
29     2.178863e-09
           ...     
454    1.762560e-09
455    1.793099e-09
456    1.799602e-09
457    1.528331e-09
458    1.139874e-09
459    1.011519e-09
460    8.878740e-10
461    1.226370e-09
462    1.634069e-09
463    9.864151e-10
464    2.113184e-10
465   -3.363933e-11
466   -4.787951e-10
467   -8.473332e-10
468   -9.969946e-10
469   -1.139061e-09
470   -1.531279e-09
471   -1.930932e-09
472   -2.068459e-09


0      2.431771e-09
1      2.426762e-09
2      2.416750e-09
3      2.406752e-09
4      2.396769e-09
5      2.386801e-09
6      2.376847e-09
7      2.366910e-09
8      2.356988e-09
9      2.207270e-09
10     2.056018e-09
11     1.991777e-09
12     1.960246e-09
13     1.904292e-09
14     1.839525e-09
15     1.807167e-09
16     1.644713e-09
17     1.458863e-09
18     1.229405e-09
19     9.618795e-10
20     9.244871e-10
21     9.659362e-10
22     6.987135e-10
23     4.718401e-10
24     4.018205e-10
25     2.580529e-10
26     2.264384e-11
27    -2.005636e-10
28    -1.896724e-10
29    -2.358182e-10
           ...     
454    3.259926e-10
455    2.485868e-10
456    1.095640e-10
457   -1.701219e-11
458    4.717953e-11
459    1.193841e-10
460    2.495699e-11
461   -6.303014e-11
462    2.524792e-11
463   -4.410897e-11
464   -7.274861e-11
465   -1.730578e-13
466    1.022249e-11
467    1.224462e-12
468   -6.199521e-11
469   -1.481505e-10
470   -2.581731e-10
471   -3.078389e-10
472   -3.908499e-10


0      2.169497e-09
1      2.165881e-09
2      2.158651e-09
3      2.151428e-09
4      2.144212e-09
5      2.137004e-09
6      2.129803e-09
7      2.122609e-09
8      2.115423e-09
9      2.064329e-09
10     1.953024e-09
11     1.828899e-09
12     1.635767e-09
13     1.341222e-09
14     1.086993e-09
15     8.959556e-10
16     7.191735e-10
17     6.098181e-10
18     4.804511e-10
19     3.436708e-10
20     2.462011e-10
21     1.522687e-10
22     7.487284e-11
23     3.764553e-11
24     7.472384e-11
25     8.095350e-11
26     1.904099e-11
27    -5.760895e-11
28    -9.892209e-11
29    -1.494068e-10
           ...     
454    5.801359e-11
455    9.330512e-11
456    1.236401e-10
457    2.402633e-10
458    2.745841e-10
459    1.994445e-10
460    1.442766e-10
461    1.531783e-10
462    1.978095e-10
463    2.023227e-10
464    1.979893e-10
465    8.773307e-11
466   -6.644847e-11
467   -9.210068e-11
468   -1.289553e-10
469   -3.154970e-10
470   -4.569124e-10
471   -5.531908e-10
472   -6.707636e-10


In [104]:
%matplotlib qt
figJ, axJ = plt.subplots(1,1)
figC, axC = plt.subplots(1,1)

t_flux_up = 50
t_flux_down = 100
c_threshold = 5e-9

def Jfit(deltac,k,c):
    return k*deltac + c

p0 = [1e-14,0]
Pparams = {}
cols = []

legend = []
for i,col in enumerate(dfJ.columns):
    

    t = dt*np.arange(0,len(dfC[col]))
    J = dfJ[col]
    J_fit = J[t_flux_up:t_flux_down]
    deltaC = dfdeltaC[col]
    deltaC_fit = deltaC[t_flux_up:t_flux_down]
    
#     deltaC_bar = np.linspace(dfC[col].min(),1)
#     params = curve_fit(Jfit,deltaC_fit,J_fit,p0)    
#     J_bar = Jfit(deltaC_fit,*params[0])
#     P = params[0][0]
#     dP = np.sqrt(params[1][0][0])
#     print(dP/P, P)
    

    
#     if  P < 0:
#         dfJ.pop(col)
        
#         dfdeltaC.pop(col)        
#         continue
        
    
#     Pparams[col] = params
#     if len(J.iloc[(deltaC < 0.75).to_numpy()]) < 1:
#         dfJ.pop(col)
        
#         dfdeltaC.pop(col)         
#         continue
        
#     if col == '10009':
#         continue
#     ####################
#     # just for this particular case need to filter out a specific trace 
#     #
#     legend.append(col)
    
# #     if len(J_arr[J_arr < 0.5 ]) >=1:
# #         continue
        
    Pparams[col] = params
#     cols.append(col)
   
#     # plot fluxes
    axJ.scatter(deltaC,J,alpha = 0.25)
#     axJ.plot(deltaC_fit,J_bar)
    
    #plot the Intensity vs time
    
    axC.plot(t,dfC[col])
 
#Flux plot params


axJ.set_xticks([0,0.5,1])
axJ.set_xlim([-0.05,1.05])
axJ.set_xticklabels([0,0.5,1],fontsize = 15)

axJ.set_xlabel('$[K]_{ext} - [K]_{in}$ [mM]',fontsize = 18)
axJ.set_yticks([0,5e-9,1e-8,1.5e-8])
axJ.set_yticklabels([0,5,10,15],fontsize = 15)

axJ.set_ylim([-1e-9,16e-9])
#  
    
axJ.set_ylabel('J  [$10^{-9}$ mol$m^{-2}s^{-1}$]',fontsize = 18)
axJ.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)

axC.set_xticks([0,20,40])
axC.set_xticklabels([0,20,40],fontsize = 15)
axC.set_xlabel('Time [mins]',fontsize = 18)
# axC.set_xlim([-2,45])
axC.set_yticks([0,0.5,1])
axC.set_yticklabels([0,0.5,1],fontsize = 15)
axC.set_ylabel('Rel I',fontsize = 18 )
axC.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)
axC.set_ylim([-0.05,1.05])

# plt.legend()
# global params



(-0.05, 1.05)

# Save permeability values


In [120]:
P_df = {'P ms^-1': [Pparams[key][0][0] for key in Pparams.keys()],
       'dP/P': [np.sqrt(Pparams[key][1][0][0])/Pparams[key][0][0] for key in Pparams.keys()],
       'A':[np.mean(dfA[key].iloc[:30]) for key in Pparams.keys()]}
   
P_identified = {key:[Pparams[key][0][0]] for key in Pparams.keys()}
P_df = pd.DataFrame(P_df)

P_df['P cms^-1'] = P_df['P ms^-1']*100

P_df.to_csv('Permeabilities_GramicidinA_3nM_20210920.csv')

In [121]:
P_df

,P ms^-1,dP/P,A,P cms^-1
0,2.147937e-10,0.840039,2303.374249,2.147937e-08
1,1.240443e-09,0.237100,2019.778415,1.240443e-07
2,2.819658e-10,0.500319,2534.566436,2.819658e-08
3,1.774359e-08,0.011232,1015.950290,1.774359e-06
4,2.679327e-08,0.026122,2458.679978,2.679327e-06
5,1.368265e-08,0.005340,3018.309665,1.368265e-06
6,5.672738e-11,19.544700,2043.694040,5.672738e-09
7,1.551951e-08,0.004125,3417.656020,1.551951e-06
8,6.480869e-10,0.326824,2205.979457,6.480869e-08
9,7.288411e-09,0.063806,3019.097165,7.288411e-07


In [113]:
P_identified = pd.DataFrame(P_identified)

# Save Flux vs deltaC data

In [114]:
dfJ.to_csv('Flux_GramA_3nM_20210920.csv')
dfdeltaC.to_csv('deltaC_GramA_3nM_20210920.csv')
P_identified.to_csv('PbyGUVId_GramA_3nM_20210920.csv')

# The relationship between J and delta c

The variation of the flux, J, with $\Delta c$ is linear. However in order to gain a good fit, we have to relax ths condition that the flux is zero when the gradient is 0. 

Near to 1mM internal concentration the calibration underestimates the internal concentration. Therefore it overestimates the concentration gradient at the point of zero flux. This develops an associated error in the y intercept and the permeability.


In [79]:
deltaCbins = np.linspace(0.35,1,40)

deltaC_arr = dfdeltaC[cols].to_numpy()

dfJoverP = dfJ[cols]

for col in dfJoverP.columns:
    dfJoverP[col] = dfJoverP[col].to_numpy()/P_identified[col].iloc[0]
    
    
J_arr = dfJoverP.to_numpy()

J_mean = []
deltaC_mean = []

for i in range(len(deltaCbins)-1):
    
    deltaC_mean.append(np.median(deltaC_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])]))
    J_mean.append(np.median(J_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])]))


In [83]:
figm,axm = plt.subplots(1,1)


axm.scatter(deltaC_mean[:-2],J_mean[:-2])
# axm.set_ylim([-1e-9,16e-9])

In [84]:
meanFluxdata = {'deltaC':deltaC_mean[:-2],'J': J_mean[:-2]}

meanFluxdata = pd.DataFrame(meanFluxdata)

meanFluxdata.to_csv('20210920_GramA3nM_MeanFluxvsdeltaC.csv')